# Dogs vs. cats image classification: Inception V3

In [ ]:
# Handle imports up-front
import os
import sys
import glob

# Silence logging messages from TensorFlow, except errors
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

# Use a specific GPU
os.environ['CUDA_VISIBLE_DEVICES']='2'

# PyPI imports
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.optimizers import SGD
from keras import regularizers
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

# Figure out if we are running on Kaggle or not, if so
# add the location of utils.py to path so we can import
path_list=os.getcwd().split(os.sep)

if path_list[1] == 'kaggle':
    sys.path.append('/kaggle/usr/lib/image_classification_functions')

# Import custom helper functions from utils.py
from image_classification_functions import prep_data
from image_classification_functions import plot_single_training_run

# Silence logging messages from TensorFlow, except errors
tf.get_logger().setLevel('ERROR')

# Limit TensorFlow's CPU usage
tf.config.threading.set_intra_op_parallelism_threads(2)
tf.config.threading.set_inter_op_parallelism_threads(2)

In [ ]:
tune_output=True
tune_inception_block=True

output_tuned_savefile='../data/models/inceptionV3_output_tuned.pkl'
inception_tuned_savefile='../data/models/inceptionV3_tuned.pkl'

## 1. Data preparation

### 1.1. Load the data paths

In [ ]:
# Decompress and organize the images
training_data_path, validation_data_path, testing_data_path=prep_data()

# Get lists of training and validation dog and cat images
training_dogs=glob.glob(f'{training_data_path}/dogs/dog.*')
training_cats=glob.glob(f'{training_data_path}/cats/cat.*')
validation_dogs=glob.glob(f'{validation_data_path}/dogs/dog.*')
validation_cats=glob.glob(f'{validation_data_path}/cats/cat.*')

### 1.2. Create training and validation datasets

In [ ]:
training_dataset=tf.keras.utils.image_dataset_from_directory(
    training_data_path,
    image_size=(256, int(256*(3/4))),
    batch_size=16
)

validation_dataset=tf.keras.utils.image_dataset_from_directory(
    validation_data_path,
    image_size=(256, int(256*(3/4))),
    batch_size=16
)

### 1.3. Create testing dataset

In [ ]:
testing_dataset=tf.keras.utils.image_dataset_from_directory(
    training_data_path,
    image_size=(256, int(256*(3/4)))
)

## 2. Model fine-tuning
### 2.1. Train output layers

In [ ]:
%%time

l1_penalty=1e-4
l2_penalty=1e-3

if tune_output == True:

    # create the base pre-trained model
    base_model=InceptionV3(weights='imagenet', include_top=False)

    # Add some output layers
    x=base_model.output
    x=GlobalAveragePooling2D()(x)
    x=Dropout(0.5)(x)
    x=Dense(
        1024,
        activation='relu',
        kernel_regularizer=regularizers.L1L2(l1=l1_penalty, l2=1e-3)
    )(x)
    x=Dense(
        256,
        activation='relu',
        kernel_regularizer=regularizers.L1L2(l1=l1_penalty, l2=1e-3)
    )(x)
    x=Dense(
        128,
        activation='relu',
        kernel_regularizer=regularizers.L1L2(l1=l1_penalty, l2=1e-3)
    )(x)
    x=Dense(
        64,
        activation='relu',
        kernel_regularizer=regularizers.L1L2(l1=l1_penalty, l2=1e-3)
    )(x)
    predictions=Dense(1, activation='sigmoid')(x)

    model=Model(inputs=base_model.input, outputs=predictions)

    # Train only the output layers by freezing all convolutional InceptionV3 layers
    for layer in base_model.layers:
        layer.trainable = False

    # Compile the model to train on binary cross-entropy loss
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['binary_accuracy'])

    # Train the model on the new data for a few epochs
    training_result=model.fit(
        training_dataset,
        validation_data=validation_dataset,
        epochs=10
    )

    # Save the results
    with open(output_tuned_savefile, 'wb') as output_file:
        pickle.dump(training_result, output_file, protocol=pickle.HIGHEST_PROTOCOL)

else:
    with open(output_tuned_savefile, 'rb') as output_file:
        training_result=pickle.load(output_file)

print()

In [ ]:
# Plot the results
plot_single_training_run(training_result).show()

### 2.2. Train inception layers

In [ ]:
# Take a look at the model and decide how many layers to freeze and how many to train
training_result.model.summary()

In [ ]:
%%time

if tune_inception_block is True:

   # Train the top 2 inception blocks, by freezing the first 249 layers
   # and leaving the rest unfrozen
   for layer in model.layers[:249]:
      layer.trainable = False
   for layer in model.layers[249:]:
      layer.trainable = True

   # Recompile the model using SGD with a low learning rate
   model.compile(
      optimizer=SGD(learning_rate=0.0001, momentum=0.9),
      loss='binary_crossentropy',
      metrics=['binary_accuracy']
   )

   # Train again
   training_result=model.fit(
      training_dataset,
      validation_data=validation_dataset,
      epochs=20
   )

    # Save the results
    with open(inception_tuned_savefile, 'wb') as output_file:
        pickle.dump(training_result, output_file, protocol=pickle.HIGHEST_PROTOCOL)

else:
    with open(inception_tuned_savefile, 'rb') as output_file:
        training_result=pickle.load(output_file)

print()

In [ ]:
# Plot the results
plot_single_training_run(training_result).show()

## 3. Model evaluation

### 3.2. Make predictions

In [ ]:
images=np.concatenate([x for x, y in validation_dataset], axis=0)
labels=np.concatenate([y for x, y in validation_dataset], axis=0)

predictions=training_result.model.predict(images)
print(f'Testing images shape: {images.shape}')
print(f'Testing labels shape: {labels.shape}')

threshold=0.5
predictions=[1 if p > threshold else 0 for p in predictions]

accuracy=accuracy_score(predictions, labels)*100
print(f'Test set accuracy: {accuracy:.1f}%')

### 3.3. Model performance

In [ ]:
# Plot the confusion matrix
cm=confusion_matrix(labels, predictions, normalize='true')
cm_disp=ConfusionMatrixDisplay(confusion_matrix=cm)
_=cm_disp.plot()

plt.title(f'Test set performance\noverall accuracy: {accuracy:.1f}%')
plt.xlabel('Predicted class')
plt.ylabel('True class')
plt.show()